# LLM Iteration 1

**Aim**

Produce a simple initial model that generates patient arrivals for the four different patient types Stroke, TIA, Complex Neurological or Other. These will be given a unique identifier. There are 10 beds in the model and patients are only admitted if there is a free bed. After remaining on the ward for 7 days they then leave the model.

**Prompt**

Code a discrete-event simulation model of a stroke ward in a hospital in python 3.10 and simpy 4.
Please code the full model specified. Do not return a simplified version. Show all code. 

All time units in the model are in days.
Each patient in the model has a unique identifier. The first patient to arrive has an identifier of 0. For each subsequent patient increment the identifier by 1.

Patients to arrive in a stroke ward that admits four different patient types: Stroke, TIA, Complex Neurological or Other. Each patient type had a different interarrival time distribution. There are 10 beds on the stroke ward. The patient should only be admitted if there is an available bed. The patient holds the bed until their length of stay has elapsed. The length of stay for all patients is 7 days. After the length of stay has elapsed, the patient leaves the model. 

All patient types must have their own generator function. 

The model should print out useful information after each event. 
The model should include a user settable run length. This defaults to 2 months. 

The interarrival time distribution and parameters of patients are dependent on the patient type. This describes the average time between required admissions. For each distribution time is measured in days. 
Stroke = Exponential: 1.2
TIA = Exponential: 9.3
Complex Neurological = Exponential: 3.6
Other = Exponential: 3.2

Numpy should be used for sampling. Each interarrival distribution should have its own numpy.random.Generator object. 


**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**LLM Output**

In [1]:
import simpy
import numpy as np

In [2]:
class StrokeWard:
    def __init__(self, env, num_beds):
        self.env = env
        self.beds = simpy.Resource(env, capacity=num_beds)
        self.patient_count = 0
        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.los_store = []
        ##################################

    def admit_patient(self, patient):
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            ##################################

            print(f'Patient {patient.id} admitted to the stroke ward at day '
                  f'{self.env.now}')
            # Length of stay for all patients is 7 days
            yield self.env.timeout(7)
            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(bed_leave_time - bed_arrival_time)
            ##################################
            print(f'Patient {patient.id} discharged from the stroke ward at '
                  f'day {self.env.now}')

class Patient:
    def __init__(self, id, patient_type):
        self.id = id
        self.patient_type = patient_type

def stroke_arrival(env, stroke_ward):
    while True:
        interarrival_time = np.random.default_rng().exponential(1.2)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Stroke")
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def tia_arrival(env, stroke_ward):
    while True:
        interarrival_time = np.random.default_rng().exponential(9.3)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "TIA")
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def complex_neurological_arrival(env, stroke_ward):
    while True:
        interarrival_time = np.random.default_rng().exponential(3.6)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Complex Neurological")
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def other_arrival(env, stroke_ward):
    while True:
        interarrival_time = np.random.default_rng().exponential(3.2)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Other")
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

In [3]:
env = simpy.Environment()
stroke_ward = StrokeWard(env, num_beds=10)

env.process(stroke_arrival(env, stroke_ward))
env.process(tia_arrival(env, stroke_ward))
env.process(complex_neurological_arrival(env, stroke_ward))
env.process(other_arrival(env, stroke_ward))

# Set the run length to 2 months (default)
run_length_days = 60 * 2

print("Simulation started...")
env.run(until=run_length_days)
print("Simulation ended.")

Simulation started...
Patient 0 admitted to the stroke ward at day 1.0258950296502078
Patient 1 admitted to the stroke ward at day 1.4327969811847832
Patient 2 admitted to the stroke ward at day 1.8544857554354395
Patient 3 admitted to the stroke ward at day 2.6033977643780597
Patient 4 admitted to the stroke ward at day 2.8574852280371266
Patient 5 admitted to the stroke ward at day 3.021570705607807
Patient 6 admitted to the stroke ward at day 3.8445011086942324
Patient 7 admitted to the stroke ward at day 4.07124887042491
Patient 8 admitted to the stroke ward at day 4.8362722633483255
Patient 9 admitted to the stroke ward at day 4.8660415437157996
Patient 0 discharged from the stroke ward at day 8.025895029650208
Patient 10 admitted to the stroke ward at day 8.025895029650208
Patient 1 discharged from the stroke ward at day 8.432796981184783
Patient 11 admitted to the stroke ward at day 8.432796981184783
Patient 2 discharged from the stroke ward at day 8.854485755435439
Patient 12 a

This Python code implements a discrete-event simulation model of a stroke ward in a hospital using SimPy 4. It includes different patient types (Stroke, TIA, Complex Neurological, Other) with specific interarrival time distributions and parameters. The model tracks patient admissions and discharges based on bed availability and length of stay. The simulation runs for a user-settable default period of 2 months.

**Checks**

In [4]:
class TestCase:
    def __init__(self, env, ward):
        self.env = env
        self.ward = ward
        self.env.process(stroke_arrival(env, ward))
        self.env.process(tia_arrival(env, ward))
        self.env.process(complex_neurological_arrival(env, ward))
        self.env.process(other_arrival(env, ward))

    def test_run_length(self):
        '''
        Checks that specified run length is implemented correctly.
        '''
        self.env.run(until=RUN_LENGTH)

        if self.env.now > RUN_LENGTH:
            print('The simulation has continued for longer than input.')

    def test_bed_hold(self):
        '''
        Checks that patients are holding beds until they leave the model.
        '''
        self.env.run(until=RUN_LENGTH)

        for bed_use in ward.bed_in_use_store:
            if bed_use > NUM_BEDS:
                print('The number of beds in use exceeded limit.')

    def test_length_of_stay(self):
        '''
        Checks that specified length of stay correctly implemented.
        '''
        self.env.run(until=RUN_LENGTH)

        for los in ward.los_store:
            if round(los, 10) > 7:
                print('Patient length of stay exceeded limit.')
            elif round(los, 10) < 7:
                print('Patient did not stay for the full duration.')

In [5]:
# Checks run length
NUM_BEDS = 10
RUN_LENGTH = 10

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward)
test.test_run_length()

Patient 0 admitted to the stroke ward at day 1.778125307611034
Patient 1 admitted to the stroke ward at day 2.7953127113623766
Patient 2 admitted to the stroke ward at day 2.931265768231965
Patient 3 admitted to the stroke ward at day 4.452334561379942
Patient 4 admitted to the stroke ward at day 4.8393561346371055
Patient 5 admitted to the stroke ward at day 6.049915726423313
Patient 6 admitted to the stroke ward at day 6.579084283332706
Patient 7 admitted to the stroke ward at day 7.359104194639829
Patient 0 discharged from the stroke ward at day 8.778125307611035
Patient 8 admitted to the stroke ward at day 9.701646899632589
Patient 1 discharged from the stroke ward at day 9.795312711362376
Patient 2 discharged from the stroke ward at day 9.931265768231965


In [6]:
# Checks that is only ever 1 patient if there is only 1 bed
NUM_BEDS = 1
RUN_LENGTH = 60

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward)
test.test_bed_hold()

Patient 0 admitted to the stroke ward at day 0.30236731754321644
Patient 0 discharged from the stroke ward at day 7.302367317543217
Patient 1 admitted to the stroke ward at day 7.302367317543217
Patient 1 discharged from the stroke ward at day 14.302367317543217
Patient 2 admitted to the stroke ward at day 14.302367317543217
Patient 2 discharged from the stroke ward at day 21.302367317543215
Patient 3 admitted to the stroke ward at day 21.302367317543215
Patient 3 discharged from the stroke ward at day 28.302367317543215
Patient 4 admitted to the stroke ward at day 28.302367317543215
Patient 4 discharged from the stroke ward at day 35.302367317543215
Patient 5 admitted to the stroke ward at day 35.302367317543215
Patient 5 discharged from the stroke ward at day 42.302367317543215
Patient 6 admitted to the stroke ward at day 42.302367317543215
Patient 6 discharged from the stroke ward at day 49.302367317543215
Patient 7 admitted to the stroke ward at day 49.302367317543215
Patient 7 dis

In [7]:
# Checks if stay length correctly implemented
NUM_BEDS = 10
RUN_LENGTH = 10

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward)
test.test_length_of_stay()

Patient 0 admitted to the stroke ward at day 0.6598693576528504
Patient 1 admitted to the stroke ward at day 0.7381742161124004
Patient 2 admitted to the stroke ward at day 2.0237960193034352
Patient 3 admitted to the stroke ward at day 3.4797801313188876
Patient 4 admitted to the stroke ward at day 5.757834917235765
Patient 5 admitted to the stroke ward at day 7.128821027793411
Patient 6 admitted to the stroke ward at day 7.6249599254538305
Patient 0 discharged from the stroke ward at day 7.65986935765285
Patient 7 admitted to the stroke ward at day 7.679339741418264
Patient 1 discharged from the stroke ward at day 7.738174216112401
Patient 2 discharged from the stroke ward at day 9.023796019303436
Patient 8 admitted to the stroke ward at day 9.442157422456505
Patient 9 admitted to the stroke ward at day 9.5807007346212


**Reflection**

The LLM has implemented code where patients of type Stroke, TIA, Complex Neurological or Other arrive at the ward. They are admitted if there is a free bed and they stay on the ward for 7 days. No more patients are admitted if the beds are full, as checked when changing the number of beds to 1, where only one patient was admitted at any one time. There is no option to change the length of stay in the code as implemented, but it has correctly set up a length of stay of 7 days.